# Artifical Intelligence Project Phase I
# Exam Schedular using Genetic Algorithms


In [20]:
import random
import pandas as pd

# Extend time slots to include weekdays
DAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
TIME_SLOTS = [f'{day} {time}' for day in DAYS for time in ['9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00']]

# Read data from CSV files
teachers_df = pd.read_csv('teachers.csv')
students_df = pd.read_csv('studentNames.csv')
courses_df = pd.read_csv('courses.csv')
student_courses_df = pd.read_csv('studentCourse.csv')

# Convert to lists or dictionaries
teachers = teachers_df['Names'].tolist()
students = students_df['Names'].tolist()
courses = dict(zip(courses_df['Course Code'], courses_df['Course Name']))
student_courses = list(zip(student_courses_df['Student Name'], student_courses_df['Course Code']))

# Define classrooms
classrooms = [f'C3{str(i).zfill(2)}' for i in range(1, 11)]

class Schedule:
    def __init__(self, data):
        self.data = data
        self.schedule = []  # List of tuples (course_code, day_time_slot, room, teacher)
        self.fitness = 0

    def initialize(self):
        """ Initialize a schedule considering all constraints. """
        courses = self.data['courses']
        teachers = self.data['teachers']
        classrooms = self.data['classrooms']
        occupied = {teacher: [] for teacher in teachers}  # Track teacher schedules

        for course in courses:
            while True:
                room = random.choice(classrooms)
                teacher = random.choice(teachers)
                day_time_slot = random.choice(TIME_SLOTS)

                if day_time_slot not in occupied[teacher]:
                    self.schedule.append((course, day_time_slot, room, teacher))
                    occupied[teacher].append(day_time_slot)
                    break
        return self

    def calculate_fitness(self):
        """ Calculate fitness by counting constraint violations. """
        self.fitness = 120  # Start with a perfect score and deduct for violations
        
        # Mapping to store time slots for each student and teacher to check for conflicts
        student_times = {student: [] for student in self.data['students']}
        teacher_times = {teacher: [] for teacher in self.data['teachers']}
        
        # Check for each entry in the schedule
        for (course, day_time, room, teacher) in self.schedule:
            day, time = day_time.split()  # Split into day and time
            hour = int(time.split(':')[0])  # Extract the hour for time checks
            
            # Check if the time is within the permissible hours (9 am to 5 pm)
            if hour < 9 or hour > 16:
                self.fitness -= 10 # Penalize for scheduling outside of permissible hours
            
            # Check for teacher's time conflict
            if day_time in teacher_times[teacher]:
                self.fitness -= 10  # Penalize for the teacher having more than one exam at the same time
            teacher_times[teacher].append(day_time)
            
            # Track all students in the course to check for conflicts
            for student, course_code in self.data['student_courses']:
                if course_code == course:
                    if day_time in student_times[student]:
                        self.fitness -= 10  # Penalize for scheduling a student for more than one exam at the same time
                    student_times[student].append(day_time)
        
        # Check if all courses have an exam scheduled
        scheduled_courses = set([entry[0] for entry in self.schedule])
        for course in self.data['courses']:
            if course not in scheduled_courses:
                self.fitness -= 40  # Penalize for each course that doesn't have a scheduled exam
        
        # Ensure a teacher doesn't invigilate two exams in a row
        for teacher, times in teacher_times.items():
            sorted_times = sorted(times)
            for i in range(len(sorted_times) - 1):
                current_day, current_time = sorted_times[i].split()
                next_day, next_time = sorted_times[i + 1].split()
                if current_day == next_day and abs(int(current_time.split(':')[0]) - int(next_time.split(':')[0])) == 1:
                    self.fitness -= 15  # Penalize for back-to-back exams by the same teacher

def crossover(parent1, parent2):
    """ Crossover two parents to create a new schedule. """
    cutoff = random.randint(1, len(parent1.schedule) - 1)
    new_schedule = Schedule(parent1.data)
    new_schedule.schedule = parent1.schedule[:cutoff] + parent2.schedule[cutoff:]
    new_schedule.calculate_fitness()  # Calculate fitness for the new schedule
    return new_schedule

def mutate(schedule, mutation_rate=0.1):
    """ Randomly mutate the schedule based on a mutation rate. """
    if random.random() < mutation_rate:  # Check if mutation should occur
        idx = random.randint(0, len(schedule.schedule) - 1)
        course, _, room, teacher = schedule.schedule[idx]
        day_time_slot = random.choice(TIME_SLOTS)
        schedule.schedule[idx] = (course, day_time_slot, room, teacher)
        schedule.calculate_fitness()  # Recalculate fitness after mutation

def genetic_algorithm(input_data, generations=10, population_size=5, mutation_rate=0.1):
    population = [Schedule(input_data).initialize() for _ in range(population_size)]
    best_schedule = None
    best_fitness = float('-inf')
    fitness_history = []

    for gen in range(generations):
        # Calculate fitness for each individual
        for individual in population:
            individual.calculate_fitness()

        # Sort population based on fitness
        population.sort(key=lambda x: x.fitness, reverse=True)

        # Capture fitness values for this generation
        current_fitness_values = [individual.fitness for individual in population]
        fitness_history.append(current_fitness_values)

        # Track the best and worst schedules
        current_best = population[0]
        current_worst = population[-1]

        # Print the best and worst fitness values and schedules of this generation
        print(f"Generation {gen + 1} Best Fitness: {current_best.fitness}")
        print(f"Generation {gen + 1} Worst Fitness: {current_worst.fitness}")
        print_schedule_details(current_worst)

        # Update best schedule if the current best is better
        if current_best.fitness > best_fitness:
            best_schedule = current_best
            best_fitness = current_best.fitness
            print(f"New best fitness found: {best_fitness} at Generation {gen + 1}")

        # Selection and evolution
        next_gen = population[:int(len(population)/2)]
        while len(next_gen) < population_size:
            parent1, parent2 = random.sample(next_gen, 2)
            child = crossover(parent1, parent2)
            mutate(child, mutation_rate)  # Apply mutation
            next_gen.append(child)
        population = next_gen

    # Summarize best findings
    print(f"Best overall fitness: {best_fitness}")
    return best_schedule, fitness_history

def print_schedule_details(schedule):
    """ Print details of a schedule """
    print("Schedule Details:")
    for course, time, room, teacher in schedule.schedule:
        print(f"Course: {course}, Time: {time}, Room: {room}, Teacher: {teacher}")

# Remaining code to initialize data and run the genetic algorithm stays the same


# Run the genetic algorithm
exam_input = {
    'students': students,
    'courses': list(courses.keys()),
    'teachers': teachers,
    'classrooms': classrooms,
    'student_courses': student_courses,
    'exam_duration': 1  # Duration in hours
}
best_schedule, fitness_history = genetic_algorithm(exam_input)
day_map = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5}

print("\nBest Schedule")
# Display and store the sorted schedule
sorted_schedule = sorted(best_schedule.schedule, key=lambda x: (day_map[x[1].split()[0]], x[1].split()[1]))
for course, time, room, teacher in sorted_schedule:
    print(f"Course: {course}, Time: {time}, Room: {room}, Teacher: {teacher}")

# Save the best schedule to CSV
schedule_data = [[course, time, room, teacher] for course, time, room, teacher in sorted_schedule]
df = pd.DataFrame(schedule_data, columns=['Course Code', 'Time', 'Classroom', 'Teacher'])
df.to_csv('best_schedule.csv', index=False





Generation 1 Best Fitness: 90
Generation 1 Worst Fitness: -10
Schedule Details:
Course: CS217, Time: Friday 16:00, Room: C310, Teacher: Sanaa Ilyas
Course: EE227, Time: Friday 14:00, Room: C308, Teacher: Maimoona Rassol
Course: CS211, Time: Thursday 11:00, Room: C302, Teacher: Irum Inayat
Course: SE110, Time: Tuesday 10:00, Room: C305, Teacher: Zainab Moin
Course: CS118, Time: Friday 13:00, Room: C305, Teacher: Amna Irum
Course: CS219, Time: Thursday 12:00, Room: C310, Teacher: Shafaq Riaz
Course: CS220, Time: Thursday 11:00, Room: C309, Teacher: Usman Ashraf
Course: CS302, Time: Monday 10:00, Room: C303, Teacher: Sadia Nauman
Course: CY2012, Time: Thursday 11:00, Room: C305, Teacher: Khadija Farooq
Course: CS307, Time: Thursday 10:00, Room: C308, Teacher: Aqeel Shahzad
Course: CS328, Time: Monday 16:00, Room: C306, Teacher: Nagina Safdar
Course: EE229, Time: Tuesday 14:00, Room: C304, Teacher: Nagina Safdar
Course: AI2011, Time: Wednesday 10:00, Room: C310, Teacher: Arshad Islam
Cours